### Ответы на вопросы по теоретической части

- А точно нужно сортировать по вероятности?

Точно. В этом и состоит наша задача - определить и обратить внимание пользователя на тот товар, который ему нужен и который он может купить. В конечном итоге наша работа и заключается в том, чтобы определить вероятность того, что пользователю понравится наша рекомендация и чем она выше - тем качественнее проделанная нами работа.

- Какую метрику использовать?

Разные метрики оценивают разные показатели. Конечно, для оценки экономической эффективности нашей деятельности следует выбрать moneyprecision, однако это взвешенная метрика и может некорректно оценивать качество нашей работы. Для оценки именно качества следует пользоваться метрикой prercision.

- Сколько раз в неделю отправляем рассылку?

Будет логичным исходить из времени, проведнному на площадке. Пользователь, который провел на нашей площадке 3 часа в поисках какого-то товара гораздо более лоялен к нам и заинтересован в покупках. Следовательно, коммуникация с ним была довольно продолжительная и, скорее всего мы ничего нового ему не сможем предложить в рассылке. Пользователь, который провел на площадке минимум времени или не заходил на нее довольно продолжительное время, вряд ли будет рад читать письма чаще, чем 2 раза в неделю. 

- В какое время отправляем рассылку?

Необходимо определить то время, в которое юзер чаще всего был длительное время онлайн. Чаще всего это свободное от работы время (обед, дорога домой и тд). В это время как раз и неплохо было бы ему напомнить о себе.

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

Можно придумать какой-то маркер, который будет помечать товары как нерелевантные (что-то вроде штрафа) и понижать веса этим товарам для конкретного юзера. Возможно, по количеству рекомендаций до этого (то есть если юзер видел этот товар уже 10 раз в рассылке, то сомнительно, что появление этого товара 11й раз склонит юзера к покупке); может быть такое, что пользователь даже и не интересовался товаром, а просто кликнул по сопутствующим товарам из любопытства; также товар может быть сезонным и тд. Критериев, по которым можно убрать товар из рекомендаций, может быть много, надо анализировать.

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

Обязательно нужно. Скорее всего пользователь интересуется не только (допустим) карманным амперметром, поэтому можно начать с ближайших товаров по дереву товаров. Можно порекомендовать что-то из соседней категории или чуть выше по дереву в каталоге. А можно взять другие товары, которые купили пользователи, в том числе купившие карманный амперметр. А еще, почему бы не порекомендовать человеку топ из топа? Например, товар, который купили 10% вообще из всех, кто зарегистрирован на площадке.
Если обобщить, то разные товары - товары из других (соседних или совсем других) категорий.

### Практическая часть

In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

from metrics import precision_at_k

In [2]:
data = pd.read_csv('../Manuals/webinar_2/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../Manuals/webinar_2/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/home/maxim/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [10]:
def user_recs(fact, reg):
    
    model = AlternatingLeastSquares(factors=fact, 
                                    regularization=reg,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=4)

    model.fit(sparse_user_item.T, show_progress=False)  # На вход item-user matrix

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

    precision_recs = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    
    return precision_recs

In [11]:
factors_res = dict()
for f in range(16, 256, 16):
    factors_res[f] = user_recs(f, 0.001)

In [12]:
factors_res

{16: 0.1642507345739471,
 32: 0.16415279138099903,
 48: 0.16709108716944174,
 64: 0.1624877571008815,
 80: 0.16493633692458376,
 96: 0.16307541625857003,
 112: 0.15915768854064644,
 128: 0.15181194906953965,
 144: 0.1506366307541626,
 160: 0.14466209598432908,
 176: 0.13898139079333988,
 192: 0.13535749265426053,
 208: 0.13026444662095985,
 224: 0.12546523016650343,
 240: 0.12664054848188053}

In [14]:
regular_res = dict()
for r in np.linspace(0.001, 2, 20):
    regular_res[r] = user_recs(48, r)

In [15]:
regular_res

{0.001: 0.1643486777668952,
 0.10621052631578948: 0.1681684622918707,
 0.21142105263157895: 0.1654260528893242,
 0.31663157894736843: 0.16620959843290892,
 0.4218421052631579: 0.16816846229187074,
 0.5270526315789473: 0.1685602350636631,
 0.6322631578947369: 0.16875612144955926,
 0.7374736842105264: 0.16914789422135162,
 0.8426842105263158: 0.16503428011753182,
 0.9478947368421052: 0.1693437806072478,
 1.0531052631578945: 0.165230166503428,
 1.1583157894736842: 0.16738491674828604,
 1.2635263157894736: 0.16875612144955926,
 1.368736842105263: 0.16885406464250735,
 1.4739473684210527: 0.16748285994123407,
 1.579157894736842: 0.1705190989226249,
 1.6843684210526315: 0.16973555337904017,
 1.789578947368421: 0.16669931439764935,
 1.8947894736842104: 0.16444662095984333,
 2.0: 0.1685602350636631}

Таким образом, максимальные значения мы получили при значениях regular = 1.5791 и factors = 48